In [1]:
import numpy as np
import numexpr as en

In [2]:
a , b = np.arange(1e6),np.arange(1e6)

In [3]:
%%time
a+1

CPU times: user 2.83 ms, sys: 3.25 ms, total: 6.08 ms
Wall time: 5.09 ms


array([1.00000e+00, 2.00000e+00, 3.00000e+00, ..., 9.99998e+05,
       9.99999e+05, 1.00000e+06])

In [4]:
%%time
en.evaluate("a+1")

CPU times: user 5.47 ms, sys: 8.79 ms, total: 14.3 ms
Wall time: 4.51 ms


array([1.00000e+00, 2.00000e+00, 3.00000e+00, ..., 9.99998e+05,
       9.99999e+05, 1.00000e+06])

In [5]:
%%time
2*a+3*b

CPU times: user 7.18 ms, sys: 6.35 ms, total: 13.5 ms
Wall time: 11.8 ms


array([0.000000e+00, 5.000000e+00, 1.000000e+01, ..., 4.999985e+06,
       4.999990e+06, 4.999995e+06])

In [6]:
%%time
en.evaluate("2*a+3*b")

CPU times: user 10.3 ms, sys: 1.08 ms, total: 11.4 ms
Wall time: 14.7 ms


array([0.000000e+00, 5.000000e+00, 1.000000e+01, ..., 4.999985e+06,
       4.999990e+06, 4.999995e+06])

In [7]:
%%time
a**b*a*b

CPU times: user 22.9 ms, sys: 3.88 ms, total: 26.8 ms
Wall time: 43.5 ms


array([ 0.,  1., 16., ..., inf, inf, inf])

In [8]:
%%time
en.evaluate("a**b*a*b")

CPU times: user 268 ms, sys: 9.53 ms, total: 277 ms
Wall time: 71.6 ms


array([ 0.,  1., 16., ..., inf, inf, inf])

In [9]:
%%time
a.dot(b)

CPU times: user 2.99 ms, sys: 2.05 ms, total: 5.04 ms
Wall time: 11.8 ms


3.3333283333349997e+17

# 线程池配置
线程在导入时产生，其数量由环境变量设置NUMEXPR_MAX_THREADS。默认的最大线程数为64。产生比计算节点上可用的虚拟核心数量更多的线程没有优势。实际上，NumExpr 仅在非常大的矩阵（> 2 ** 22）上才能以大线程数（> 8）进行缩放。生成大量线程不是免费的，并且会增加NumExpr或导入该线程的包（例如Pandas或PyTables）的导入时间。

如果需要，可以通过环境变量NUMEXPR_NUM_THREADS（首选）或来调整使用的池中的线程数OMP_NUM_THREADS。通常，仅设置NUMEXPR_MAX_THREADS就足够了；所使用的线程数可以通过进行动态调整numexpr.set_num_threads(int)。线程数不能超过设置的数量NUMEXPR_MAX_THREADS。

如果用户在导入NumExpr之前未配置环境，则将生成信息日志，并且已使用的线程的初始数量_被设置为系统中检测到的内核数或8，以较小者为准。

用法：

In [10]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '8'
import numexpr as ne

A = a>a.mean()
A

array([False, False, False, ...,  True,  True,  True])

# 临时设置线程操作
设置要在操作中使用的线程数。返回线程数的先前设置。请参阅下面的注释，以了解如何通过环境变量设置线程数。

如果使用的是VML，则可能要使用set_vml_num_threads（nthreads）来执行与VML的并行作业。但是，使用VML优化的函数应该会获得非常相似的性能，并且VML的并行器不能处理（x + 1）*（x-2）之类的常见表达式，而NumExpr的并行表达式可以。

In [11]:
en.set_num_threads(8)

8

In [12]:
en.evaluate("a**b*a*b")

array([ 0.,  1., 16., ..., inf, inf, inf])

NumExpr支持以下列出的一组运算符：

逻辑运算符： &, |, ~
比较运算符： <, <=, ==, !=, >=, >
一元算术运算符： -
二进制算术运算符： +, -, *, /, **, %, <<, >>

In [13]:
ne.evaluate("sin(a)")

array([ 0.        ,  0.84147098,  0.90929743, ...,  0.21429647,
       -0.70613761, -0.97735203])

## 检测系统上的内核数。

In [14]:
en.detect_number_of_cores()

8

# 查表操作

In [15]:
ne.evaluate("where(True,A,b)"),ne.evaluate("where(False,A,b)")

(array([False, False, False, ...,  True,  True,  True]),
 array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 9.99997e+05,
        9.99998e+05, 9.99999e+05]))

## –三角正弦，余弦或正切。

In [16]:
en.evaluate('sin(a)'),en.evaluate("cos(a)"),en.evaluate("tan(a)")

(array([ 0.        ,  0.84147098,  0.90929743, ...,  0.21429647,
        -0.70613761, -0.97735203]),
 array([ 1.        ,  0.54030231, -0.41614684, ..., -0.97676866,
        -0.70807463,  0.21161996]),
 array([ 0.        ,  1.55740772, -2.18503986, ..., -0.21939327,
         0.99726439, -4.61843033]))

## 三角反正弦，余弦或正切。

In [17]:
en.evaluate("arcsin(a)"),en.evaluate("arccos(a)"),en.evaluate("arctan(a)")

(array([0.        , 1.57079633,        nan, ...,        nan,        nan,
               nan]),
 array([1.57079633, 0.        ,        nan, ...,        nan,        nan,
               nan]),
 array([0.        , 0.78539816, 1.10714872, ..., 1.57079533, 1.57079533,
        1.57079533]))

## float1 / float2的三角反正切。

In [18]:
en.evaluate("arctan2(a,b)")

array([0.        , 0.78539816, 0.78539816, ..., 0.78539816, 0.78539816,
       0.78539816])

## 双曲反正弦，余弦或正切。

In [19]:
en.evaluate("arcsinh(a)"),en.evaluate("arccosh(a)"),en.evaluate("arctanh(a)")

(array([ 0.        ,  0.88137359,  1.44363548, ..., 14.50865474,
        14.50865574, 14.50865674]),
 array([        nan,  0.        ,  1.3169579 , ..., 14.50865474,
        14.50865574, 14.50865674]),
 array([ 0., inf, nan, ..., nan, nan, nan]))

## 自然，以10为底和log（1 + x）对数

In [20]:
en.evaluate("log(a)"),en.evaluate("log10(a)"),en.evaluate("log1p(a)")

(array([       -inf,  0.        ,  0.69314718, ..., 13.81550756,
        13.81550856, 13.81550956]),
 array([      -inf, 0.        , 0.30103   , ..., 5.9999987 , 5.99999913,
        5.99999957]),
 array([ 0.        ,  0.69314718,  1.09861229, ..., 13.81550856,
        13.81550956, 13.81551056]))

## 指数和指数减一。

In [21]:
en.evaluate("exp(a)"),en.evaluate("expm1(a)")

(array([1.        , 2.71828183, 7.3890561 , ...,        inf,        inf,
               inf]),
 array([0.        , 1.71828183, 6.3890561 , ...,        inf,        inf,
               inf]))

## 平方根

In [22]:
en.evaluate("sqrt(a)")

array([  0.        ,   1.        ,   1.41421356, ..., 999.9985    ,
       999.999     , 999.9995    ])

## 绝对值。

In [23]:
en.evaluate("abs(a)")

array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 9.99997e+05,
       9.99998e+05, 9.99999e+05])

## 共轭值

In [24]:
en.evaluate("conj(a)")

array([0.00000e+00-0.j, 1.00000e+00-0.j, 2.00000e+00-0.j, ...,
       9.99997e+05-0.j, 9.99998e+05-0.j, 9.99999e+05-0.j])

## 复数的实部或虚部。

In [25]:
en.evaluate("real(a)"),en.evaluate("imag(a)")

(array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 9.99997e+05,
        9.99998e+05, 9.99999e+05]),
 array([0., 0., 0., ..., 0., 0., 0.]))

## 由实部和虚部组成

In [26]:
en.evaluate("complex(a,b)")

array([0.00000e+00+0.00000e+00j, 1.00000e+00+1.00000e+00j,
       2.00000e+00+2.00000e+00j, ..., 9.99997e+05+9.99997e+05j,
       9.99998e+05+9.99998e+05j, 9.99999e+05+9.99999e+05j])

## 包含的每个字符串，返回True

In [27]:
en.evaluate("contains('a,b,c','a,b,c')")

array(True)

## 给定轴上的数组元素的总和。不支持负轴

In [28]:
B = np.arange(100).reshape(10,10)
"竖轴",en.evaluate("sum(B, axis=0)"),"横轴",en.evaluate("sum(B, axis=1)")

('竖轴',
 array([450, 460, 470, 480, 490, 500, 510, 520, 530, 540], dtype=int64),
 '横轴',
 array([ 45, 145, 245, 345, 445, 545, 645, 745, 845, 945], dtype=int64))

## 给定轴上数组元素的乘积。不支持负轴

In [29]:
"竖轴",en.evaluate("prod(B, axis=0)"),"横轴",en.evaluate("prod(B, axis=1)")

('竖轴',
 array([                0,   478015854767451,  1242688846823424,
         2394832584543399,  4060162871525376,  6393838623046875,
         9585618768101376, 13865696119905399, 19511273389031424,
        26853950884211451], dtype=int64),
 '横轴',
 array([                  0,        335221286400,      72684900288000,
           2306992893004800,   29820834418636800,  227991539359584000,
        1233909752121446400, 5227941748053024000,    5769992716765184,
        1193853638847869952], dtype=int64))

## 重新评估最后一个数组表达式，而不进行任何检查。这是为了加速循环，该循环将重复地重新计算相同的表达式，而不更改操作数。如果不确定，请使用更安全的valuate（）。

In [30]:
en.re_evaluate(local_dict=None)

array([                  0,        335221286400,      72684900288000,
          2306992893004800,   29820834418636800,  227991539359584000,
       1233909752121446400, 5227941748053024000,    5769992716765184,
       1193853638847869952], dtype=int64)

In [31]:
en.test()

..-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Numexpr version:   2.7.1
NumPy version:     1.18.1
Python version:    3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Platform:          darwin-x86_64-Darwin Kernel Version 19.6.0: Thu Jun 18 20:49:00 PDT 2020; root:xnu-6153.141.1~1/RELEASE_X86_64
VML available?     True
VML/MKL version:   Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications
Number of threads used by default: 8 (out of 8 detected cores)
Maximum number of threads: 64
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
..............................................................................................................................................................................................................................................................................................................................

<unittest.runner.TextTestResult run=5522 errors=0 failures=1>

In [32]:
en.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Numexpr version:   2.7.1
NumPy version:     1.18.1
Python version:    3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Platform:          darwin-x86_64-Darwin Kernel Version 19.6.0: Thu Jun 18 20:49:00 PDT 2020; root:xnu-6153.141.1~1/RELEASE_X86_64
VML available?     True
VML/MKL version:   Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications
Number of threads used by default: 8 (out of 8 detected cores)
Maximum number of threads: 64
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [33]:
en.get_vml_version()

'Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications'

## 将使用E. <variable>变量构建的表达式编译为函数。

ex也可以指定为字符串“ 2 * a + 3 * b”。

可以使用签名参数指定输入变量及其类型的顺序，签名参数是（名称，类型）对的列表。

返回包含已编译函数的NumExpr对象。

In [34]:
n = ne.NumExpr("a+b")

## 给定一个NumExpr对象，返回一个列表，该列表是被反汇编的程序。

In [35]:
ne.disassemble(n)

[(b'add_ddd', b'r0', b'r1[a]', b'r2[b]')]

## 检测到的（虚拟）核心数。

In [36]:
ne.ncores

8

## 当前正在使用的线程数

In [37]:
ne.nthreads

8

## 由环境变量设置的最大线程数

In [38]:
ne.MAX_THREADS

64